# User Input for a Prometheus Run
=================================

Please run each code block and follow the instructions.
The end result will be a downloadable .csv file with the necessary values.
Ensure that you have python and wget installed on your machine before starting.

There is an accompanying how-to in the "Installations" tab.

In [11]:
# Necessary imports 
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import ipywidgets as widgets

from ipyleaflet import Map, Marker, GeoJSON
from ipywidgets import Output
from IPython.display import display
from datetime import datetime


In [2]:
# run the etl_nfdf_polygons.py script to get the NFDF polygons
%run etl_nfdb_polygons.py

Shapefile already exists. Downloading the latest data...


In [ ]:
# Load the GeoJSON file with geopandas
gdf = gpd.read_file('Canada_perimeters.geojson')

# Ensure CRS is WGS84
gdf = gdf.to_crs(epsg=4326)

# Optionally drop columns you don't need
gdf = gdf.drop(columns=['FIRSTDATE', 'LASTDATE', 'CONSIS_ID'])

# Convert to GeoJSON dict
geojson_data = json.loads(gdf.to_json())

# Center the map on the centroid of all polygons
center = gdf.geometry.unary_union.centroid.coords[0][::-1]
m = Map(center=center, zoom=4)

# Add the GeoJSON layer
geojson_layer = GeoJSON(
    data=geojson_data,
    name='NFDB Polygons',
    style={
        'color': 'red',
        'weight': 1,
        'fillColor': 'orange',
        'fillOpacity': 0.6
    }
)
m.add_layer(geojson_layer)

# add marker that will be updated on click
marker = Marker(location=center)    
m.add_layer(marker)

out = Output()

def handle_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        latlon = kwargs.get('coordinates')
        marker.location = latlon
        with out:
            print(f"Clicked location: {latlon}")

m.on_interaction(handle_map_click)

print("Click on the map to get coordinates")
print("Double-click to zoom in or use the zoom controls")
print("Your final click is the selected location")
display(m, out)

C:\Users\lbuchart\AppData\Local\Temp\ipykernel_35980\626565808.py:14: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gdf.geometry.unary_union.centroid.coords[0][::-1]


Map(center=[56.408567936484076, -106.80156636094897], controls=(ZoomControl(options=['position', 'zoom_in_text…

Output()

In [7]:
coords = marker.location
print(f"The final coordinates you have selected are: {coords}")

# save the map to an HTML file
m.save('nfdb_polygons_map.html')


The final coordinates you have selected are: [52.23588183145654, -107.9553115005652]


In [23]:
# now select model and dates to download ECCC data
import ipywidgets as widgets
from IPython.display import display
model_dropdown = widgets.Dropdown(
    options=['rdps', 'hrdps'],
    value='rdps',
    description='Model:',
)

model_run = widgets.Dropdown(
    options=['00', '06', '12', '18'],
    value='00',
    description='Model Initialization Time:',
)

In [24]:
current_year = datetime.now().year
year_slider = widgets.IntSlider(
    value=int(current_year),
    min=2020,
    max = int(current_year),
    step=1,
    description='Year:',
    continuous_update=False
)

month_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=12,
    step=1,
    description='Month:',
    continuous_update=False
)

day_slider = widgets.IntSlider(
    value=1,
    min=1, 
    max=31,
    step=1,
    description='Day:',
    continuous_update=False
)

forecast_length = widgets.IntSlider(
    value=48,
    min=24,
    max=72,
    step=6,
    description='Forecast Length (hours):',
    continuous_update=False
)

# Display the widgets
display(model_dropdown, model_run, year_slider, month_slider, day_slider, forecast_length)

print("Once you are happy with your selections")
print("Move to the next cell to begin the download process")

print("Note that the MSC Datamart only has data from the previous 30 days")

Dropdown(description='Model:', options=('rdps', 'hrdps'), value='rdps')

Dropdown(description='Model Initialization Time:', options=('00', '06', '12', '18'), value='00')

IntSlider(value=2025, continuous_update=False, description='Year:', max=2025, min=2020)

IntSlider(value=1, continuous_update=False, description='Month:', max=12, min=1)

IntSlider(value=1, continuous_update=False, description='Day:', max=31, min=1)

IntSlider(value=48, continuous_update=False, description='Forecast Length (hours):', max=72, min=24, step=6)

Once you are happy with your selections
Move to the next cell to begin the download process
Note that the MSC Datamart only has data from the previous 30 days


In [29]:
# function to set files names
def set_filenames():
    model = model_dropdown.value
    run = model_run.value
    year = year_slider.value
    month = month_slider.value
    day = day_slider.value
    forecast_length = forecast_length.value

    print(f"Selected Model: {model}")
    print(f"Selected Model Run: {run}")
    print(f"Selected Date: {year}-{month:02d}-{day:02d}")
    print(f"Forecast Length: {forecast_length} hours")

    # Create the filename based on the selections
    if model == 'rdps':
        # load the rdps_vars.json file
        with open('rdps_vars.json', 'r') as f:
            model_vars = json.load(f)
    elif model == 'hrdps':
        # load the hrdps_vars.json file
        with open('hrdps_vars.json', 'r') as f:
            model_vars = json.load(f)
        
    for hh in range(0, forecast_length):
        extension = f"{year}{month:02d}{day:02d}/WXO-DD/model_{model}/{model_vars['configuration']['resolution']}/{run}/0{hh:02d}/"
            
        for var in model_vars['wx_vars'].values():
            file = f"{year}{month:02d}{day:02d}T{run}Z_MSC_{model.upper()}_{var}_RLatLon{model_vars['configuration']['grid']}_PT0{hh}H.grib2"

            print(extension + file)